In [1]:
%load_ext autoreload
%autoreload 2

import tree
import simulation as sim
import sim_utils as sut
import count_model as cmod
import tree_utils as tu

In [7]:
import numpy as np
from numpy import random

In [4]:
rseed = 42
random.seed(rseed)

In [5]:
top = [["A", "B"], ["B", "C"], ["B", "D"], ["D", "E"], ["C", "F"]]

In [8]:
branches = np.unique(np.array(top).flatten())
time = {b: 30 for b in branches}
G = 500
t = tree.Tree(topology=top, G=G, time=time, num_branches=len(branches), branch_points=1, modules=40)

idea: have parallel branches that have a forced % of correlated expr programs and then have normal branches from their ends